In [4]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/hsuehchen/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


In [14]:
mars_news_url = 'https://mars.nasa.gov/news/'
browser.visit(mars_news_url)
soup = BeautifulSoup(browser.html, 'html.parser')


In [19]:
# retrieving latest news page
soup.find_all('div', class_='slide')

[]